In [1]:
#Assignment 1
import cv2

In [ ]:
#Queston 1
"The calibration matrix from performing the camera calibration using Matlab's Computer Vision Toolbox is defined below."

k = [[1341.88563669822,	0,	1024.14808411592],
     [0,	1349.70698899083,	609.383415036218],
     [0,	0,	1]]



In [ ]:
# Question 2


In [2]:
# Question 3
import cv2
import depthai as dai
from object_detector import *
import numpy as np

# Load Aruco detector
parameters = cv2.aruco.DetectorParameters()
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_50)

# Load Object Detector
detector = HomogeneousBgDetector()

# Pipeline initialization
pipeline = dai.Pipeline()

# Define the DepthAI camera
camRgb = pipeline.create(dai.node.ColorCamera)
xout = pipeline.create(dai.node.XLinkOut)

xout.setStreamName('video')

# Properties
camRgb.setBoardSocket(dai.CameraBoardSocket.RGB)
camRgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
camRgb.setFps(30)

# Linking
camRgb.video.link(xout.input)

# Start the pipeline
with dai.Device(pipeline) as device:

    # Output queue will be used to get the video frames from the output defined above
    q = device.getOutputQueue(name="video", maxSize=30, blocking=True)

    while True:
        # Get DepthAI camera frame
        frame = q.get().getCvFrame()

        # Get Aruco marker
        corners, _, _ = cv2.aruco.detectMarkers(frame, aruco_dict, parameters=parameters)
        if corners:

            # Draw polygon around the marker
            int_corners = np.int0(corners)
            cv2.polylines(frame, int_corners, True, (0, 255, 0), 5)

            # Aruco Perimeter
            aruco_perimeter = cv2.arcLength(corners[0], True)

            # Pixel to cm ratio
            pixel_cm_ratio = aruco_perimeter / 20

            contours = detector.detect_objects(frame)

            # Draw objects boundaries
            for cnt in contours:
                # Get rect
                rect = cv2.minAreaRect(cnt)
                (x, y), (w, h), angle = rect

                # Get Width and Height of the Objects by applying the Ratio pixel to cm
                object_width = w / pixel_cm_ratio
                object_height = h / pixel_cm_ratio

                # Display rectangle
                box = cv2.boxPoints(rect)
                box = np.int0(box)

                cv2.circle(frame, (int(x), int(y)), 5, (0, 0, 255), -1)
                cv2.polylines(frame, [box], True, (255, 0, 0), 2)
                cv2.putText(frame, "Width {} cm".format(round(object_width, 1)), (int(x - 100), int(y - 20)), cv2.FONT_HERSHEY_PLAIN, 2, (100, 200, 0), 2)
                cv2.putText(frame, "Height {} cm".format(round(object_height, 1)), (int(x - 100), int(y + 15)), cv2.FONT_HERSHEY_PLAIN, 2, (100, 200, 0), 2)

        cv2.imshow("Image", frame)
        key = cv2.waitKey(1)  
        if key == ord('q'):  # 'q' key is pressed
            break

cv2.destroyAllWindows()


C:\Users\azuma\AppData\Local\Temp\ipykernel_13300\1886558798.py:24: DeprecationWarning: RGB is deprecated, use CAM_A or address camera by name instead.
  camRgb.setBoardSocket(dai.CameraBoardSocket.RGB)
C:\Users\azuma\AppData\Local\Temp\ipykernel_13300\1886558798.py:46: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  int_corners = np.int0(corners)
C:\Users\azuma\AppData\Local\Temp\ipykernel_13300\1886558798.py:69: DeprecationWarning: `np.int0` is a deprecated alias for `np.intp`.  (Deprecated NumPy 1.24)
  box = np.int0(box)
